In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pmdarima as pm
from statsmodels.tsa.stattools import adfuller
import pmdarima as pm

from dbfread import DBF
from pandas import DataFrame


In [ ]:
# 2045 forecast
ylist = ['yr'+str(x) for x in list(range(2015, 2046, 5))]
dfxls = pd.read_excel('semcog_2045_RDF_community_final.xlsx', sheet_name=ylist)

dfxlss = []
for y in ylist:
    df = dfxls[y][['city_id', 'hh']].set_index('city_id')
    df.columns = [y]
    dfxlss.append(df)

dfhh_2045 = pd.concat(dfxlss, axis=1)
dfhh_2045 = dfhh_2045.loc[(dfhh_2045.index>1000) | (dfhh_2045.index<=5)]

In [ ]:
dfhh_2045_pct = 100 * dfhh_2045.divide(dfhh_2045['yr2020'], axis='index') - 100
dfhh_2045_pct.columns = list(range(2015,2046,5))
dfhh_2045_pct = dfhh_2045_pct[list(range(2025, 2046, 5))]
dfhh_2045_pct.index.name = 'semmcd'
dfhh_2045_pct.head(2) # 2025 to 2045 every 5 year

In [ ]:
dfhh_2045_pct.columns

In [ ]:
#historical data hh 2000 to 2020
hhs = pd.read_csv('sem_hist_2000_2020.csv',parse_dates=['year']).set_index('year')
hhs_trans = pd.crosstab(hhs.semmcd,hhs.index.year, hhs.hh, aggfunc ='sum')
hhs_trans.head(2)

In [ ]:
# arcgis based, combine all time series 
dic_cols = {'FCAST_'+ str(c): 2020 + c for c in list(range(1,11))}
arcgis_methods = {'gompertz':'hh_gom.dbf', 'exponential': 'hh_exp.dbf', 'random_forest':'hh_forest.dbf', 'parabolic': 'hh_para.dbf', 'exponential smoothing': 'hh_expsmooth.dbf'}

dfs = []
for m in arcgis_methods.keys():
    df = DataFrame(iter(DBF(arcgis_methods[m])))
    df.rename(columns= {'SEMMCD': 'semmcd'}, inplace=True)
    df['method'] = df.METHOD.str.partition(';')[0]
    df.rename(columns = dic_cols, inplace=True)
    dfs.append(df[['semmcd', 'method'] + list(dic_cols.values())])
df_arcgis = pd.concat(dfs, axis=0)

df_arcgis = pd.merge(hhs_trans, df_arcgis, left_index=True, right_on='semmcd', how='right').set_index(['semmcd', 'method'])

In [ ]:
df_arcgis.head(2)

In [ ]:
# other time series forecasts combined
dict_py = {'arima': 'hh_forecast_arima.csv', 
'sarimax': 'pred_by_mcd_year_sarimax.csv',
'poly': 'pred_by_mcd_year_poly.csv',
'nn': 'pred_by_mcd_year_nn.csv',
'autoreg': 'pred_by_mcd_year_autoreg.csv',
'arma': 'pred_by_mcd_year_arma.csv'}

dfs = []
for p in dict_py.keys():
    df = pd.read_csv(dict_py[p], index_col = [0])
    df.index.name = 'semmcd'
    df['method'] = p
    dfs.append(df)

df_py = pd.concat(dfs, axis=0).reset_index()
df_py = df_py.set_index(['semmcd', 'method'])

df_py.columns = [int(c) for c in df_py.columns]

In [ ]:
df_all = pd.concat([df_arcgis, df_py], axis=0)

In [ ]:
df_test = df_all.reset_index().set_index('semmcd')
df_test['base'] = hhs_trans[2017]
df_test.set_index('method', append=True, inplace=True)


In [ ]:
df_test2  = df_test.divide(df_test['base'], axis='index') *100 - 100
df_test2_pct = df_test2[list(range(2025, 2046, 5))]

In [ ]:

df_test2  = df_test.divide(df_test['base'], axis='index') *100 - 100
df_test2_pct = df_test2[list(range(2025, 2046, 5))]

In [ ]:
### test here MSE? or other methods 
### compare df_test2_pct and dfhh_2045_pct

In [ ]:
df_test2_pct.index.get_level_values(0).unique()
df_test2_pct = df_test2_pct.reset_index().set_index('semmcd')
df_test2_pct.sub(dfhh_2045_pct[[2030]])

In [ ]:
dd_tt3 = df_test2_pct[[2030]] - dfhh_2045_pct[[2030]]

In [ ]:
dd_tt3.loc[dd_tt3.loc[5, level].groupby(level=0).col2.idxmin()]

In [ ]:
dd_tt3.xs(5, level=0).idxmin()
#df.xs('price', level=1, drop_level=False)

In [ ]:
dd_tt3.xs(4005, level=0)

In [ ]:
dd_tt3

In [ ]:
df.loc[df.loc[df.col1 == 5.].groupby(level=0).col2.idxmin()]

In [ ]:
df_test2_pct = df_test2[list(range(2025, 2046, 5))]

In [ ]:
dfhh_2045_pct.columns = []

In [ ]:
df_py.loc[5].columns

In [ ]:
for t in dict_tx.keys():
    

#hhs.index = hhs.index.year
hhs.dtypes

In [ ]:
hhs_trans = pd.crosstab(hhs.semmcd,hhs.index.year, hhs.hh, aggfunc ='sum')

In [ ]:
hhs_trans

In [ ]:
def arimamodel(timeseries):
    automodel = pm.auto_arima(timeseries, 
                              start_p=1, 
                              start_q=1,
                              test="adf",
                              seasonal=False,
                              trace=True)
    return automodel
def plotarima(n_periods, timeseries, automodel):
    # Forecast
    fc, confint = automodel.predict(n_periods=n_periods, 
                                    return_conf_int=True)
    # Weekly index
    fc_ind = pd.date_range(timeseries.index[timeseries.shape[0]-1], 
                           periods=n_periods, freq="Y")
    # Forecast series
    fc_series = pd.Series(fc, index=fc_ind)
    # Upper and lower confidence bounds
    lower_series = pd.Series(confint[:, 0], index=fc_ind)
    upper_series = pd.Series(confint[:, 1], index=fc_ind)
    # Create plot
    plt.figure(figsize=(10, 6))
    plt.plot(timeseries)
    plt.plot(fc_series, color="red")
    plt.xlabel("date")
    plt.ylabel(timeseries.name)
    plt.fill_between(lower_series.index, 
                     lower_series, 
                     upper_series, 
                     color="k", 
                     alpha=0.25)
    plt.legend(("past", "forecast", "95% confidence interval"),  
               loc="upper left")
    plt.show()

In [ ]:
years = 30
fc_years = pd.date_range(hhs.index.max(), periods=years + 1, freq="Y").year[1:]

fcs = []
ids = []
for sid in hhs.semmcd.unique():
    tsd =  hhs.loc[hhs.semmcd==sid]
    tsdata = tsd.hh
    automodel = arimamodel(tsdata)
    print(tsd.semmcd.iloc[0],tsd.mcdname.iloc[0] )
    fc = automodel.predict(n_periods= years, return_conf_int=False)
    fcs.append(fc)
    ids.append(sid)
fc_output = pd.DataFrame(fcs, columns =list(fc_years), index=ids)

In [ ]:
fc_output = pd.DataFrame(fcs, columns =list(fc_years), index=ids)

In [ ]:
fc_output

In [ ]:
hh_forecast_arima = pd.concat([hhs_trans,fc_output], axis=1).astype(int)

In [ ]:
hh_forecast_arima.to_csv('hh_forecast_arima.csv')

In [ ]:
hh_forecast_arima

In [ ]:
fc_output_pct = (1 - fc_output.divide(hhs_trans[2020].astype(float), axis=0)).round(3)

In [ ]:
fc_output_pct.to_csv('hh_pct_forecast_arima.csv')

In [ ]:
hhsaa = hhs.loc[hhs.semmcd==4005]

hhsaa = hhsaa.set_index('year')
tsdata=hhsaa['hh']


In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
print("p-value:", adfuller(hhsaa['hh'])[1])

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax1 = fig.add_subplot(311)
fig = plot_acf(tsdata, ax=ax1,
               title="Autocorrelation on Original Series") 
ax2 = fig.add_subplot(312)
fig = plot_acf(tsdata.diff().dropna(), ax=ax2, 
               title="1st Order Differencing")
ax3 = fig.add_subplot(313)
fig = plot_acf(tsdata.diff().diff().dropna(), ax=ax3, 
               title="2nd Order Differencing")

In [ ]:
plot_pacf(tsdata.diff().dropna(), lags=10)

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
model = ARIMA(tsdata, order=(1, 1, 1))
results = model.fit()
results.summary()
results.plot_predict(1, 20)

In [ ]:
results.plot_predict(1, 30)

In [ ]:
import statsmodels
statsmodels.__version__

In [ ]:
hhs = pd.read_csv('sem_hist_2000_2020.csv',parse_dates=['year']).set_index('year')
hhs.index = hhs.index.year
hhs.dtypes

hhstrans = pd.crosstab(hhs.semmcd,hhs.index, hhs.hh, aggfunc ='sum')


In [ ]:
import pmdarima as pm

In [ ]:
def arimamodel(timeseries):
    automodel = pm.auto_arima(timeseries, 
                              start_p=1, 
                              start_q=1,
                              test="adf",
                              seasonal=False,
                              trace=True)
    return automodel
def plotarima(n_periods, timeseries, automodel):
    # Forecast
    fc, confint = automodel.predict(n_periods=n_periods, 
                                    return_conf_int=True)
    # Weekly index
    fc_ind = pd.date_range(timeseries.index[timeseries.shape[0]-1], 
                           periods=n_periods, freq="Y")
    # Forecast series
    fc_series = pd.Series(fc, index=fc_ind)
    # Upper and lower confidence bounds
    lower_series = pd.Series(confint[:, 0], index=fc_ind)
    upper_series = pd.Series(confint[:, 1], index=fc_ind)
    # Create plot
    plt.figure(figsize=(10, 6))
    plt.plot(timeseries)
    plt.plot(fc_series, color="red")
    plt.xlabel("date")
    plt.ylabel(timeseries.name)
    plt.fill_between(lower_series.index, 
                     lower_series, 
                     upper_series, 
                     color="k", 
                     alpha=0.25)
    plt.legend(("past", "forecast", "95% confidence interval"),  
               loc="upper left")
    plt.show()

In [ ]:
years = 30
fc_years = pd.date_range(hhs.year.max(), periods=years, freq="Y").year[1:]

fcs = []
for sid in hhs.semmcd.unique():
    tsd =  hhs.loc[hhs.semmcd==sid]
    tsdata = tsd.hh
    automodel = arimamodel(tsdata)
    print(tsd.semmcd.iloc[0],tsd.semmcd_name.iloc[0] )
    fc = automodel.predict(n_periods= years, return_conf_int=False)
    fcs.append(fc)
    ids.append(sid)
fc_output = pd.DataFrame(fcs, columns =list(fc_years), index=ids)



In [ ]:
years = 30
tsd =  hhs.loc[hhs.semmcd==4005]
tsdata = tsd.hh
automodel = arimamodel(tsdata)

In [ ]:
fc = automodel.predict(n_periods= years, return_conf_int=False)

In [ ]:
fc_years = pd.date_range(hhs.year.max(), periods=years, freq="Y").year[1:]

In [ ]:
pd.crosstab(hhs.semmcd, hhs.year)

In [ ]:
fc_years

In [ ]:
len(fc)

In [ ]:
s1 = pd.Series(fc,index =fc_years, name =id )

In [ ]:
s2 = pd.Series(fc, name =id+1 )

In [ ]:
pd.DataFrame([s1,s2])

In [ ]:
pd.DataFrame([fc, fc], columns =list(fc_years), index=[3,4])

In [ ]:
for id in hhs.semmcd.unique():
    print(id)

In [ ]:
years

In [ ]:
years = 30
dtime = pd.date_range(tsdata.index[tsdata.shape[0]-1], 
                           periods=years, freq="Y")
dtime = pd.DatetimeIndex(dtime).year #only keep year
automodel = arimamodel(tsdata)

fc = automodel.predict(n_periods= years, return_conf_int=False)


In [ ]:
pd.date_range(tsdata.index[tsdata.shape[0]-1], periods=years, freq="Y").year

In [ ]:
ndf = pd.DataFrame(columns= dtime)

In [ ]:
ndf.append(pd.Series(fc), ignore_index=True)

In [ ]:
pd.DataFrame([dtime, fc])

In [ ]:
dtime

In [ ]:
fc = automodel.predict(n_periods=20, return_conf_int=False)

In [ ]:
fc

In [ ]:
jobs = jobs.set_index('year')

In [ ]:
jobs

In [ ]:
for id in [ 4005, 4010, 4015, 4020, 4025, 4030, 4035, 4040, 4045, 4050, 4055]:
    tsd =  jobs.loc[jobs.semmcd==id]
    tsdata = tsd.jobs
    automodel = arimamodel(tsdata)
    print(tsd.semmcd.iloc[0],tsd.semmcd_name.iloc[0] )
    plotarima(20, tsdata, automodel)
    

In [ ]:
automodel = arimamodel(jobaa["jobs"])
#plotarima(70, jobaa["jobs"], automodel)

In [ ]:
automodel.summary()

In [ ]:
plotarima(20, jobaa["jobs"], automodel)